# Naive portfolio strategies - comparisons

The following "naive" portfolios are considered:

- equal weighted portfolio (on the rebalancing date, the weights are proportional to 1/N, where N is the number of portfolio components).
- inverse volatility (on the rebalancing date, the weights are proportional to the inverse of the portfolio component volatilities, experienced during a predefined historical lookback period).
- inverse variance (on the rebalancing date, the weights are proportional to the inverse of the portfolio component variances, experienced during a predefined historical lookback period).
- inverse drawdown (on the rebalancing date, the weights are proportional to the asset maximum historical drawdwon, observed over a predefined historical lookback period).

We start by importing the **azapy** and other useful packages.

In [1]:
import sys
sys.path.append("..")
import azapy as az

### Collect historical market data

Note the flag `force=False`. In this case the function will attempt to read the market data from `mktdir`. If it fails, then it will access the data provider servers, in this case *yahoo*. For more information see https://azapy.readthedocs.io/en/latest/.

In [2]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

sdate = "2012-01-01"
edate = "2021-07-27"

mktdir = "../MkTdata"

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, file_dir=mktdir)

read GLD data from file
read TLT data from file
read XLV data from file
read VGT data from file
read PSJ data from file

Request between 2012-01-01 : 2021-07-27
                    GLD         TLT         XLV         VGT         PSJ
source            yahoo       yahoo       yahoo       yahoo       yahoo
force             False       False       False       False       False
save               True        True        True        True        True
file_dir     ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
verbose            True        True        True        True        True
error                No          No          No          No          No
nrow               2407        2407        2407        2407        2407
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2021-07-27  2021-07-27  2021-07-27

### Set the model parameters

Since we are interested to run several portfolio strategies, it is convenient to setup the model parameters in a dictionary. 
The keys are given by the portfolio names while the model parameters are encapsulated in a dictionary:

- `type` is the name of the portfolio class, 
- `m_param` is a dictionary of parameters to be set by `set_model` method.

The `hlength`is set to 3.25 years. This is the length of historical lookback period used to calibrate the historical volatilities, variances and maximum drawdowns.

For `P_N`, the `ww` is set to `None`, indicating the default setup for equal weighted portfolio. 

*For more details see the documentations for Port_InvVol, Port_InvVar, Port_InvDD and Port_ConstW classes.*

In [3]:
models = {'P_InvVol': {'type': 'Port_InvVol', 'm_param': {'hlength': 3.25}},
          'P_InvVaR': {'type': 'Port_InvVar', 'm_param': {'hlength': 3.25}},
          'P_InvDD': {'type': 'Port_InvDD', 'm_param': {'hlength': 3.25}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### Main computational loop

- `port` is the list of portfolio time-series.
- `pp` is a list of portfolio objects. They could be interrogated later for specific information.

In [4]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

### Prepare the results for comparisons 

Build a `Port_Simple` class holding all the computed portfolios as components. We use this structure to build comparative graphical and numerical performance reports. The aggregated portfolio of portfolios time-series will be neglected.

>Note the call to `set_model` method that is a must.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [5]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Visualize the portfolio time-series

Note the use of the flags
- `componly=True` to retain only the original portfolios time-series and to exclude the aggregated portfolio of portfolios time-series,
- `fancy=True` to use the `plotly` interactive time series package.

*For more details see the documentation.*

In [10]:
_ = ps.port_view_all(componly=True, fancy=True, title="Relative performance")

### Portfolio performances 

We are using the same flags as above.

In this case the `P_InvDD` is the best performer. It also has the highest beta with higher rate of returns and smaller maximum drawdown than the equal weighted portfolio. 

In [7]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_InvDD,15.60,-17.08,0.913076,2020-03-18,2020-02-19,2020-04-27
P_InvVaR,12.41,-14.76,0.840571,2020-03-18,2020-02-19,2020-04-16
P_InvVol,13.30,-16.03,0.829984,2020-03-18,2020-02-19,2020-04-16
P_N,14.24,-17.43,0.816747,2020-03-18,2020-02-19,2020-04-24


### Portfolio annual returns

We are using the following flags:
- `withcom=True` to include the components in the report,
- `componly=True` to exclude the aggregated portfolio, 
- `fancy=True` to report the rates in percent.

The P_InvDD outperforms P_N almost every year. The exception is 2017 where the difference in the rate of returns is 25bps (bases points).

In [8]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_InvDD,P_InvVaR,P_InvVol,P_N
year,,,,
2015,-1.42%,-1.94%,-1.81%,-1.59%
2016,8.87%,6.80%,7.55%,8.21%
2017,22.57%,23.55%,23.20%,23.12%
2018,7.08%,2.63%,2.70%,2.52%
2019,28.45%,24.44%,25.21%,25.77%
2020,25.99%,18.95%,21.68%,25.54%
2021,5.93%,2.44%,3.89%,4.90%


### Historical weights for P_InvDD

>Remark: At present the GLD weight is very small (less than 2%).

In [9]:
pp['P_InvDD'].get_weights(fancy=True)

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,14.23,8.16,19.63,22.11,35.87
1,2015-09-25,2015-09-24,2012-06-22,14.60,24.76,13.46,20.08,27.11
2,2015-12-28,2015-12-24,2012-09-24,13.65,28.45,3.10,16.46,38.34
3,2016-03-28,2016-03-24,2012-12-24,10.26,20.68,16.15,24.09,28.82
4,2016-06-27,2016-06-24,2013-03-22,11.63,23.43,6.21,27.30,31.43
5,2016-09-27,2016-09-26,2013-06-26,20.92,20.96,5.56,24.43,28.13
6,2016-12-27,2016-12-23,2013-09-23,18.66,17.11,21.35,19.93,22.95
7,2017-03-28,2017-03-27,2013-12-27,15.26,18.21,22.72,19.39,24.43
8,2017-06-27,2017-06-26,2014-03-26,9.83,19.37,24.17,20.63,25.99
9,2017-09-26,2017-09-25,2014-06-25,18.60,17.87,22.30,17.25,23.98
